<img src="https://upload.wikimedia.org/wikipedia/commons/c/c7/HEIG-VD_Logo_96x29_RVB_ROUGE.png" alt="HEIG-VD Logo" width="250"/>

# TB - DevOps: Mise en place de cloud-native storage
## fio benchmark
La version de fio utilisée pour chaque benchmark va être explicitement décrite tel que la license morale (section 4 [documentation](https://fio.readthedocs.io/en/latest/fio_doc.html#moral-license) fio) nous recommande.

## Local
Les résultats de cette section ont été généré à l'aide de fio version 3.16 à l'aide du script `run-all-jobs.sh`.

In [1]:
import json

job_names = ['randomread', 'randomwrite', 'sequentialread']
job_outputs_filename = [f"{job}.fio-output" for job in job_names]

In [2]:
job_outputs = []

# Load the results
for job_output in job_outputs_filename:
    f = open(f"fio-jobs/{job_output}")
    data = json.load(f)
    job_outputs.append(data)

print(job_outputs[0])

{'fio version': 'fio-3.16', 'timestamp': 1624715980, 'timestamp_ms': 1624715980268, 'time': 'Sat Jun 26 15:59:40 2021', 'global options': {'bs': '4K', 'iodepth': '256', 'direct': '1', 'ioengine': 'libaio', 'runtime': '120', 'numjobs': '4', 'size': '512M', 'name': 'raw-readread', 'rw': 'randread'}, 'jobs': [{'jobname': 'job1', 'groupid': 0, 'error': 0, 'eta': 0, 'elapsed': 121, 'job options': {'filename': '/dev/nvme0n1p7'}, 'read': {'io_bytes': 199040774144, 'io_kbytes': 194375756, 'bw_bytes': 1658617830, 'bw': 1619743, 'iops': 404935.9938, 'runtime': 120004, 'total_ios': 48593939, 'short_ios': 0, 'drop_ios': 0, 'slat_ns': {'min': 728, 'max': 3734933, 'mean': 1513.3561, 'stddev': 1458.146573}, 'clat_ns': {'min': 51367, 'max': 12383842, 'mean': 2526780.637604, 'stddev': 589803.058981, 'percentile': {'1.000000': 733184, '5.000000': 2056192, '10.000000': 2146304, '20.000000': 2211840, '30.000000': 2277376, '40.000000': 2342912, '50.000000': 2408448, '60.000000': 2473984, '70.000000': 26050

## Cluster IICT
TODO option directory=???

On souhaite déployer un container docker avec un PV dans le cluster IICT. Le container contient l'outil *fio* et un dossier *fio-jobs*.

Ayant signé la clause de confidentialité, on ne vas pas publier le repo git

`# docker build --tag fiobench . && docker run -d fiobench`
